In [ ]:
# Mount Google Drive (for saving models)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/AndreaLolli2912/SemEval2026-EmoVA.git
%cd SemEval2026-EmoVA

Cloning into 'SemEval2026-EmoVA'...
remote: Enumerating objects: 495, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 495 (delta 102), reused 72 (delta 27), pack-reused 321 (from 1)
Receiving objects: 100% (495/495), 1.62 MiB | 5.32 MiB/s, done.
Resolving deltas: 100% (262/262), done.
/content/SemEval2026-EmoVA


In [ ]:
import os
import torch
import json
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import types
from scipy.stats import pearsonr

# Import your modules
from src.data.dataset import EmoVADataset
from src.data.collate import create_collate_fn
from src.models import AffectModel
from src.models.tokenizer_wrapper import TokenizerWrapper
from src.training import train_epoch, GradientClipper

In [ ]:
!pwd

/content/SemEval2026-EmoVA


In [ ]:
run_folder = '/content/drive/MyDrive/SEMEVAL2026_EMOVA/model_checkpoints/20260128_153120_score0.5920'

print(f"Loading checkpoint from: {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location='cpu')

config_data = checkpoint['config']
if isinstance(config_data, dict):
    config = SimpleNamespace(**config_data)
else:
    config = config_data

print(f"Config loaded. Model: {config.model_name}, LoRA: {getattr(config, 'lora', False)}")

'''config_path = f"{run_folder}/config.json"
print(f"Loading config from: {config_path}")

with open(config_path, 'r') as f:
    config_dict = json.load(f)

# Convert dict to Namespace
config = types.SimpleNamespace(**config_dict)'''

Loading config from: /content/drive/MyDrive/SEMEVAL2026_EMOVA/model_checkpoints/20260128_153120_score0.5920/config.json


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if hasattr(config, 'seed'):
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.seed)

In [ ]:
tokenizer = TokenizerWrapper(config.model_name, config.max_text_length)
full_dataset = EmoVADataset(config.data_path)
collate_fn = create_collate_fn(tokenizer)
train_loader = DataLoader(
    full_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=config.num_workers
)

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
'''model = AffectModel(
    model_path=config.model_name,
    encoder_bitfit=config.encoder_bitfit,
    isab_inducing_points=config.isab_inducing_points,
    pma_num_seeds=config.pma_num_seeds,
    lstm_hidden_dim=config.lstm_hidden_dim,
    lstm_num_layers=config.lstm_num_layers,
    lstm_bidirectional=True,
    dropout=config.dropout,
    constrain_output=False,
)

if config.encoder_bitfit:
    model.encoder.backbone.gradient_checkpointing_enable()

model = model.to(device)'''

In [ ]:
from src.models import AffectModel
from peft import LoraConfig, get_peft_model, TaskType

# 1. Inizializza Modello Base
print(f"Initializing base model: {config.model_name}")
model = AffectModel(
    model_path=config.model_name,
    encoder_bitfit=getattr(config, 'encoder_bitfit', False),
    pma_num_seeds=config.pma_num_seeds,
    isab_inducing_points=config.isab_inducing_points,
    n_heads=config.n_heads,
    lstm_hidden_dim=config.lstm_hidden_dim,
    lstm_num_layers=config.lstm_num_layers,
    lstm_bidirectional=True,
    dropout=0.0, # Zero dropout for inference
    constrain_output=getattr(config, 'constrain_output', False),
)

# 2. Applica LoRA (Se usato nel training)
use_lora = getattr(config, 'lora', False)
if use_lora:
    print("Applying LoRA structure for weight loading...")
    peft_config = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        inference_mode=True,
        r=8, lora_alpha=32, lora_dropout=0.1, target_modules=["query", "value"]
    )
    # Tentativo di applicazione specifico (come in training) o globale
    try:
        model.encoder.backbone = get_peft_model(model.encoder.backbone, peft_config)
    except Exception:
        model = get_peft_model(model, peft_config)

# 3. Carica i Pesi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("Loading state dict...")
try:
    # strict=False è fondamentale per LoRA salvati custom
    keys = model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    print(f"Loaded. Missing keys: {len(keys.missing_keys)} (Expected if base model keys are skipped)")
except Exception as e:
    print(f"Error loading weights: {e}")

model.eval()
print("Model ready!")

In [ ]:
# Optimizer with separate LRs
param_groups = [
    {'params': list([p for n, p in model.encoder.named_parameters() if p.requires_grad]),
     'lr': 5e-6, 'name': 'encoder_bias'},
    {'params': list(model.isab.parameters()), 'lr': config.lr, 'name': 'isab'} if model.isab else None,
    {'params': list(model.pma.parameters()), 'lr': config.lr, 'name': 'pma'},
    {'params': list(model.lstm.parameters()), 'lr': config.lr, 'name': 'lstm'},
    {'params': list(model.head.parameters()), 'lr': config.lr, 'name': 'head'},
]

param_groups = [
    pg for pg in param_groups
    if pg is not None and len(pg['params']) > 0
]

optimizer = AdamW(param_groups, weight_decay=config.weight_decay)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
clipper = GradientClipper(max_norm=config.max_grad_norm)

for pg in optimizer.param_groups:
    n_params = sum(p.numel() for p in pg['params'])
    print(f"{pg.get('name', 'unnamed')}: {n_params:,} params, lr={pg['lr']:.1e}")

In [ ]:
for epoch in range(config.epochs):
    # train_epoch returns dict with 'loss'
    result = train_epoch(
        model, train_loader, config.loss, optimizer, device, config, clipper=clipper
    )
    print(f"   Epoch {epoch+1}/{config.epochs} | Loss: {result['loss']:.4f}")

In [ ]:
model.eval()

predictions = {}
gold = {}

with torch.no_grad():
    for batch in tqdm(train_loader, desc="Inferencing"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        seq_lengths = batch['seq_lengths'].to(device)
        seq_mask = batch['seq_attention_mask'].to(device)

        # Targets
        valences = batch['valences'].to(device)
        arousals = batch['arousals'].to(device)
        user_ids = batch['user_ids']

        # Forward
        preds = model(input_ids, attention_mask, seq_lengths, seq_mask)

        # Move to CPU
        preds = preds.cpu().numpy()
        valences = valences.cpu().numpy()
        arousals = arousals.cpu().numpy()

        # Organize by User
        for i, uid in enumerate(user_ids):
            valid_len = seq_lengths[i].item()

            # Slice valid sequence
            p_seq = preds[i, :valid_len, :]
            t_val = valences[i, :valid_len]
            t_aro = arousals[i, :valid_len]

            # Stack targets [Seq, 2]
            t_seq = np.stack([t_val, t_aro], axis=-1)

            if uid not in predictions:
                predictions[uid] = []
                gold[uid] = []

            predictions[uid].append(p_seq)
            gold[uid].append(t_seq)

# Concatenate if users were split across batches (rare with shuffle=True but possible)
for uid in predictions:
    predictions[uid] = np.concatenate(predictions[uid], axis=0)
    gold[uid] = np.concatenate(gold[uid], axis=0)

In [ ]:
user_pred_v = np.array([predictions[u][:, 0].mean() for u in predictions.keys()])
user_pred_a = np.array([predictions[u][:, 1].mean() for u in predictions.keys()])
user_gold_v = np.array([gold[u][:, 0].mean() for u in gold.keys()])
user_gold_a = np.array([gold[u][:, 1].mean() for u in gold.keys()])

r_val, _ = pearsonr(user_gold_v, user_pred_v)
r_aro, _ = pearsonr(user_gold_a, user_pred_a)

results = {
    'valence/r_between': r_val,
    'arousal/r_between': r_aro
}

print(f"Training Fit (Between-User): Valence={r_val:.3f}, Arousal={r_aro:.3f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Valence
axes[0].scatter(user_gold_v, user_pred_v, alpha=0.5, s=15, c='blue')
axes[0].plot([-2, 2], [-2, 2], 'r--', label='Perfect', linewidth=2)
axes[0].set_xlabel('Gold Valence (User Mean)')
axes[0].set_ylabel('Predicted Valence (User Mean)')
axes[0].set_title(f"Valence - Between User (r={results['valence/r_between']:.3f})")
axes[0].legend()
axes[0].grid(True, alpha=0.3)


# Arousal
axes[1].scatter(user_gold_a, user_pred_a, alpha=0.5, s=15, c='green')
axes[1].plot([0, 2], [0, 2], 'r--', label='Perfect', linewidth=2)
axes[1].set_xlabel('Gold Arousal (User Mean)')
axes[1].set_ylabel('Predicted Arousal (User Mean)')
axes[1].set_title(f"Arousal - Between User (r={results['arousal/r_between']:.3f})")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Load test data
test_path = '/content/drive/MyDrive/SEMEVAL2026_EMOVA/dataset/TEST_RELEASE_5JAN2026/test_subtask1.csv'
test_df = pd.read_csv(test_path)

# Sort by user and timestamp (same as training)
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df = test_df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)

print(f"Test set: {len(test_df)} texts, {test_df['user_id'].nunique()} users")

In [ ]:
model.eval()
results_list = []

results_list = []

with torch.no_grad():
    # Use your preferred groupby loop
    for user_id, group in tqdm(test_df.groupby('user_id', sort=False), desc="Predicting"):
        texts = group['text'].tolist()
        text_ids = group['text_id'].tolist()

        # Tokenize
        tokenized = tokenizer(texts)
        input_ids = tokenized['input_ids'].unsqueeze(0).to(device)  # [1, S, T]
        attention_mask = tokenized['attention_mask'].unsqueeze(0).to(device)
        seq_lengths = torch.tensor([len(texts)], device=device)
        seq_mask = torch.ones(1, len(texts), device=device)

        # Predict
        preds = model(input_ids, attention_mask, seq_lengths, seq_mask)  # [1, S, 2]
        preds = preds[0].cpu().numpy()  # [S, 2]

        # Store results
        for i, text_id in enumerate(text_ids):
            results_list.append({
                'user_id': user_id,
                'text_id': text_id,
                'valence': preds[i, 0],
                'arousal': preds[i, 1],
            })

# --- 3. SAVE & ZIP ---
submission_df = pd.DataFrame(results_list)

# Verify
print("\nPreview:")
print(submission_df.head())

# Save
os.chdir('/content')
submission_df.to_csv('pred_subtask1.csv', index=False)

# Create Zip
!zip -j submission.zip pred_subtask1.csv
!unzip -l submission.zip

In [ ]:
os.chdir('/content')  # or /kaggle/working

submission_df.to_csv('pred_subtask1.csv', index=False)
!zip -j submission.zip pred_subtask1.csv

!unzip -l submission.zip